In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import trimesh
import argparse
import os
import time
import numpy as np
import torch
import torch.optim as optim
import lib.utils as utils
from lib.utils import standard_normal_logprob
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular
import pandas as pd
import random
from lib.visualize_flow import visualize_transform
import lib.layers.odefunc as odefunc
from matplotlib import pyplot as plt
%matplotlib inline 
import glob

In [2]:
SOLVERS = ["dopri5"]
parser = argparse.ArgumentParser('SoftFlow')
parser.add_argument(
    '--data', choices=['2spirals_1d','2spirals_2d', 'swissroll_1d','swissroll_2d', 'circles_1d', 'circles_2d', '2sines_1d', 'target_1d'],
    type=str, default='3d_microgel'
)
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=["concatsquash"])
parser.add_argument('--dims', type=str, default='64-64-64')
parser.add_argument("--num_blocks", type=int, default=1, help='Number of stacked CNFs.')
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True)
parser.add_argument("--divergence_fn", type=str, default="brute_force", choices=["brute_force", "approximate"])
parser.add_argument("--nonlinearity", type=str, default="tanh", choices=odefunc.NONLINEARITIES)

parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)

parser.add_argument('--residual', type=eval, default=False, choices=[True, False])
parser.add_argument('--rademacher', type=eval, default=False, choices=[True, False])
parser.add_argument('--spectral_norm', type=eval, default=False, choices=[True, False])
parser.add_argument('--niters', type=int, default=100000)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--test_batch_size', type=int, default=1000)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=1e-5)

# for the proposed method
parser.add_argument('--std_min', type=float, default=0.0)
parser.add_argument('--std_max', type=float, default=0.1)
parser.add_argument('--std_weight', type=float, default=2)

parser.add_argument('--viz_freq', type=int, default=100)
parser.add_argument('--val_freq', type=int, default=400)
parser.add_argument('--log_freq', type=int, default=10)
parser.add_argument('--gpu', type=int, default=0)
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
# logger
args.data = "microgel_multiclass"
save_path = './results/' + args.data + '/SoftFlow'
utils.makedirs(save_path)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
def get_transforms(model):

    def sample_fn(z, logpz=None):
        zeors_std = torch.zeros(z.shape[0], 1).to(z)
        if logpz is not None:
            return model(z, zeors_std, logpz, reverse=True)
        else:
            return model(z, zeors_std, reverse=True)

    def density_fn(x, logpx=None):
        zeors_std = torch.zeros(x.shape[0], 1).to(x)
        if logpx is not None:
            return model(x, zeors_std, logpx, reverse=False)
        else:
            return model(x, zeors_std, reverse=False)

    return sample_fn, density_fn

DATA_add = "/home/staff/azad/deeplearning/pointcloud/PAINT_DiffTemp/Core-shell/"
Type     = ['Microgel_plotter_v1_density', 'Microgel_plotter_v2_solvatochromism']

temperature_list = os.listdir(DATA_add)
temperature_list.sort()
temperature_list.remove('35C')

def get_data():
    idt = random.randint(0,len(temperature_list)-1)
    DATA_list = glob.glob(DATA_add+ temperature_list[idt]+'/'+Type[0]+"/*.3d")
    temperature = int(temperature_list[0][-3:-1])
    indx = random.randint(0,len(DATA_list)-1)
    DATA = pd.read_csv(DATA_list[indx], delimiter = "\t", header=None)
    DATA = DATA.to_numpy()[:,:4]
    DATA -= np.mean(DATA, axis=0) 
    DATA = (DATA-np.min(DATA, axis=0))/(np.max(DATA, axis=0)-np.min(DATA, axis=0)) 
    newrow = [temperature, temperature, temperature, temperature]
    DATA = np.vstack([newrow, DATA])

    return DATA



def compute_loss(args, model, batch_size=None):
    if batch_size is None: batch_size = args.batch_size
    x = get_data()


    x = torch.from_numpy(x).type(torch.float32).to(device)
    zero = torch.zeros(x.shape[0], 1).to(x)

    # transform to z
    std = (args.std_max - args.std_min) * torch.rand_like(x[:,0]).view(-1,1) + args.std_min
    eps = torch.randn_like(x) * std
    std_in = std / args.std_max * args.std_weight
#     print(x.shape, std_in.shape, zero.shape)
    z, delta_logp = model(x+eps, std_in, zero)

    # compute log q(z)
    logpz = standard_normal_logprob(z).sum(1, keepdim=True)

    logpx = logpz - delta_logp
    loss = -torch.mean(logpx)
    return loss

In [5]:
model = build_model_tabular(args, 4).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

time_meter = utils.RunningAverageMeter(0.93)
loss_meter = utils.RunningAverageMeter(0.93)
nfef_meter = utils.RunningAverageMeter(0.93)
nfeb_meter = utils.RunningAverageMeter(0.93)
tt_meter = utils.RunningAverageMeter(0.93)

end = time.time()
best_loss = float('inf')

In [6]:
LOW = -4
HIGH = 4
def visualize3d(samples, prior_sample, prior_logdensity, n_samples, transform, inverse_transform, memory=100, title1="$x ~ p(x)$", title2="$q(x)$", title3="$x ~ q(x)$", device="cpu", npts = 100):
    ## first one
    fig = plt.figure(figsize=(25,5))
    ax1 = fig.add_subplot(1, 3, 1, projection="3d")
    ax1.scatter(samples[:, 0], samples[:, 1], samples[:, 2], s=1, c='b', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title1)
    
    ## second one
    side = np.linspace(LOW, HIGH, npts)
    xx, yy, zz = np.meshgrid(side, side, side)
    memory=100
    x = np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1), zz.reshape(-1, 1)])
    x = torch.from_numpy(x).type(torch.float32).to(device)
    zeros = torch.zeros(x.shape[0], 1).to(x)

    z, delta_logp = [], []
    inds = torch.arange(0, x.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        z_, delta_logp_ = inverse_transform(x[ii], zeros[ii])
        z.append(z_)
        delta_logp.append(delta_logp_)
    z = torch.cat(z, 0)
    delta_logp = torch.cat(delta_logp, 0)

    logpz = prior_logdensity(z).view(z.shape[0], -1).sum(1, keepdim=True)  # logp(z)
    logpx = logpz - delta_logp

    px = np.exp(logpx.detach().cpu().numpy()).reshape(npts, npts,npts)
    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(px[:,:,1])
    ax.set_title(title2)
    
    ## Third one
    z = prior_sample(n_samples, 3).type(torch.float32).to(device)
    zk = []
    inds = torch.arange(0, z.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        zk.append(transform(z[ii]))

    zk = torch.cat(zk, 0).detach().cpu().numpy()

    ax1 = fig.add_subplot(1, 3, 3, projection="3d")
    ax1.scatter(zk[:, 0], zk[:, 1], zk[:, 2], s=1, c='r', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title3)


In [ ]:
model.train()
for itr in range(1, args.niters + 1):
    optimizer.zero_grad()

    loss = compute_loss(args, model)
    

    loss_meter.update(loss.item())

    total_time = count_total_time(model)
    nfe_forward = count_nfe(model)

    loss.backward()
    optimizer.step()

    nfe_total = count_nfe(model)
    nfe_backward = nfe_total - nfe_forward
    nfef_meter.update(nfe_forward)
    nfeb_meter.update(nfe_backward)

    time_meter.update(time.time() - end)
    tt_meter.update(total_time)

    log_message = (
        'Iter {:04d} | Time {:.4f}({:.4f}) | Loss {:.6f}({:.6f}) | NFE Forward {:.0f}({:.1f})'
        ' | NFE Backward {:.0f}({:.1f}) | CNF Time {:.4f}({:.4f})'.format(
            itr, time_meter.val, time_meter.avg, loss_meter.val, loss_meter.avg, nfef_meter.val, nfef_meter.avg,
            nfeb_meter.val, nfeb_meter.avg, tt_meter.val, tt_meter.avg
        )
    )

    print(log_message)
    
    if itr % args.val_freq == 0 or itr == args.niters:
        with torch.no_grad():
            model.eval()
            test_loss = compute_loss(args, model, batch_size=args.test_batch_size)
            test_nfe = count_nfe(model)
            log_message = '[TEST] Iter {:04d} | Test Loss {:.6f} | NFE {:.0f}'.format(itr, test_loss, test_nfe)
            print(log_message)

            if test_loss.item() < best_loss:
                best_loss = test_loss.item()
                utils.makedirs(save_path)
                torch.save({
                    'args': args,
                    'state_dict': model.state_dict(),
                }, os.path.join(save_path, 'checkpt_solv2.pth'))
            model.train()

#     if itr % args.viz_freq == 0:
#         with torch.no_grad():
#             model.eval()
# #             p_samples = toy_data.inf_train_gen(args.data, batch_size=2000)
#             p_samples = get_data()

#             sample_fn, density_fn = get_transforms(model)
#             p_samples = p_samples[1:]
#             plt.figure(figsize=(9, 3))
#             visualize3d(p_samples, torch.randn, standard_normal_logprob, p_samples.shape[0], sample_fn,density_fn, device=device)
#             fig_filename = os.path.join(save_path, 'figs', '{:04d}.jpg'.format(itr))
#             utils.makedirs(os.path.dirname(fig_filename))
#             plt.savefig(fig_filename, format='png')#, dpi=1200)
#             plt.close()
#             model.train()

    end = time.time()

print('Training has finished.')

Iter 0001 | Time 0.9906(0.9906) | Loss 4.091216(4.091216) | NFE Forward 26(26.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0002 | Time 0.5655(0.9608) | Loss 4.150503(4.095366) | NFE Forward 20(25.6) | NFE Backward 20(25.6) | CNF Time 0.5000(0.5000)
Iter 0003 | Time 0.2450(0.9107) | Loss 4.427250(4.118598) | NFE Forward 20(25.2) | NFE Backward 26(25.6) | CNF Time 0.5000(0.5000)
Iter 0004 | Time 0.2270(0.8629) | Loss 4.368974(4.136124) | NFE Forward 20(24.8) | NFE Backward 20(25.2) | CNF Time 0.5000(0.5000)
Iter 0005 | Time 0.2224(0.8180) | Loss 4.235088(4.143052) | NFE Forward 20(24.5) | NFE Backward 20(24.9) | CNF Time 0.5000(0.5000)
Iter 0006 | Time 0.2051(0.7751) | Loss 4.549300(4.171489) | NFE Forward 20(24.2) | NFE Backward 20(24.5) | CNF Time 0.5000(0.5000)
Iter 0007 | Time 0.2104(0.7356) | Loss 4.067760(4.164228) | NFE Forward 20(23.9) | NFE Backward 20(24.2) | CNF Time 0.5000(0.5000)
Iter 0008 | Time 0.2619(0.7024) | Loss 4.157099(4.163729) | NFE Forward 20(23.6) | 

Iter 0064 | Time 0.2727(0.3302) | Loss 3.506099(3.662119) | NFE Forward 26(28.0) | NFE Backward 26(27.3) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 0.3588(0.3322) | Loss 2.971974(3.613809) | NFE Forward 26(27.8) | NFE Backward 26(27.2) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 0.3311(0.3322) | Loss 3.003439(3.571083) | NFE Forward 32(28.1) | NFE Backward 32(27.6) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 0.3221(0.3314) | Loss 3.132122(3.540356) | NFE Forward 32(28.4) | NFE Backward 32(27.9) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 0.3606(0.3335) | Loss 2.972504(3.500606) | NFE Forward 38(29.1) | NFE Backward 32(28.2) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 0.3396(0.3339) | Loss 3.482917(3.499368) | NFE Forward 38(29.7) | NFE Backward 32(28.4) | CNF Time 0.5000(0.5000)
Iter 0070 | Time 0.3454(0.3347) | Loss 2.622416(3.437981) | NFE Forward 38(30.3) | NFE Backward 32(28.7) | CNF Time 0.5000(0.5000)
Iter 0071 | Time 0.3451(0.3354) | Loss 2.742623(3.389306) | NFE Forward 38(30.8) | 

Iter 0127 | Time 0.4486(0.4242) | Loss -0.292532(0.264713) | NFE Forward 50(45.9) | NFE Backward 44(43.6) | CNF Time 0.5000(0.5000)
Iter 0128 | Time 0.4493(0.4259) | Loss -0.917440(0.181962) | NFE Forward 50(46.2) | NFE Backward 44(43.6) | CNF Time 0.5000(0.5000)
Iter 0129 | Time 0.4578(0.4282) | Loss -1.035060(0.096770) | NFE Forward 38(45.6) | NFE Backward 50(44.1) | CNF Time 0.5000(0.5000)
Iter 0130 | Time 0.4579(0.4303) | Loss -0.973868(0.021826) | NFE Forward 50(45.9) | NFE Backward 44(44.1) | CNF Time 0.5000(0.5000)
Iter 0131 | Time 0.4981(0.4350) | Loss -1.065906(-0.054315) | NFE Forward 50(46.2) | NFE Backward 50(44.5) | CNF Time 0.5000(0.5000)
Iter 0132 | Time 0.3867(0.4316) | Loss -0.426700(-0.080382) | NFE Forward 38(45.6) | NFE Backward 44(44.5) | CNF Time 0.5000(0.5000)
Iter 0133 | Time 0.4285(0.4314) | Loss -0.034636(-0.077180) | NFE Forward 50(45.9) | NFE Backward 44(44.4) | CNF Time 0.5000(0.5000)
Iter 0134 | Time 0.4984(0.4361) | Loss -0.662584(-0.118158) | NFE Forward

Iter 0189 | Time 0.4908(0.5180) | Loss -1.385452(-1.337177) | NFE Forward 50(46.8) | NFE Backward 50(57.3) | CNF Time 0.5000(0.5000)
Iter 0190 | Time 0.5317(0.5190) | Loss -2.470430(-1.416505) | NFE Forward 50(47.0) | NFE Backward 56(57.2) | CNF Time 0.5000(0.5000)
Iter 0191 | Time 0.4632(0.5151) | Loss -0.949366(-1.383805) | NFE Forward 44(46.8) | NFE Backward 50(56.7) | CNF Time 0.5000(0.5000)
Iter 0192 | Time 0.5195(0.5154) | Loss -2.286975(-1.447027) | NFE Forward 56(47.4) | NFE Backward 50(56.2) | CNF Time 0.5000(0.5000)
Iter 0193 | Time 0.4992(0.5142) | Loss -1.155764(-1.426639) | NFE Forward 50(47.6) | NFE Backward 50(55.8) | CNF Time 0.5000(0.5000)
Iter 0194 | Time 0.5244(0.5150) | Loss -2.135616(-1.476267) | NFE Forward 56(48.2) | NFE Backward 50(55.4) | CNF Time 0.5000(0.5000)
Iter 0195 | Time 0.4994(0.5139) | Loss -1.718509(-1.493224) | NFE Forward 50(48.3) | NFE Backward 50(55.0) | CNF Time 0.5000(0.5000)
Iter 0196 | Time 0.4810(0.5116) | Loss -2.143801(-1.538764) | NFE For

Iter 0251 | Time 0.5201(0.5027) | Loss -2.366028(-1.749138) | NFE Forward 56(49.3) | NFE Backward 50(51.3) | CNF Time 0.5000(0.5000)
Iter 0252 | Time 0.4985(0.5024) | Loss -1.527293(-1.733609) | NFE Forward 50(49.4) | NFE Backward 50(51.2) | CNF Time 0.5000(0.5000)
Iter 0253 | Time 0.5082(0.5028) | Loss -1.169132(-1.694096) | NFE Forward 56(49.8) | NFE Backward 50(51.1) | CNF Time 0.5000(0.5000)
Iter 0254 | Time 0.5176(0.5039) | Loss -2.338071(-1.739174) | NFE Forward 50(49.9) | NFE Backward 56(51.4) | CNF Time 0.5000(0.5000)
Iter 0255 | Time 0.4418(0.4995) | Loss -2.412386(-1.786299) | NFE Forward 44(49.4) | NFE Backward 50(51.3) | CNF Time 0.5000(0.5000)
Iter 0256 | Time 0.4438(0.4956) | Loss -2.354737(-1.826089) | NFE Forward 44(49.1) | NFE Backward 50(51.3) | CNF Time 0.5000(0.5000)
Iter 0257 | Time 0.4440(0.4920) | Loss -2.368846(-1.864082) | NFE Forward 44(48.7) | NFE Backward 50(51.2) | CNF Time 0.5000(0.5000)
Iter 0258 | Time 0.5511(0.4961) | Loss -2.387395(-1.900714) | NFE For

...

Iter 15489 | Time 0.7329(0.7560) | Loss -1.624742(-2.414379) | NFE Forward 62(61.2) | NFE Backward 86(85.2) | CNF Time 0.5000(0.5000)
Iter 15490 | Time 0.7313(0.7543) | Loss -1.480757(-2.349026) | NFE Forward 62(61.3) | NFE Backward 86(85.3) | CNF Time 0.5000(0.5000)
Iter 15491 | Time 0.7334(0.7528) | Loss -1.918029(-2.318856) | NFE Forward 62(61.3) | NFE Backward 86(85.3) | CNF Time 0.5000(0.5000)
Iter 15492 | Time 0.6995(0.7491) | Loss -1.854546(-2.286354) | NFE Forward 56(61.0) | NFE Backward 86(85.4) | CNF Time 0.5000(0.5000)
Iter 15493 | Time 0.6976(0.7455) | Loss -2.588390(-2.307497) | NFE Forward 56(60.6) | NFE Backward 86(85.4) | CNF Time 0.5000(0.5000)
Iter 15494 | Time 0.7541(0.7461) | Loss -2.637391(-2.330589) | NFE Forward 62(60.7) | NFE Backward 86(85.5) | CNF Time 0.5000(0.5000)
Iter 15495 | Time 0.7972(0.7496) | Loss -3.203301(-2.391679) | NFE Forward 62(60.8) | NFE Backward 86(85.5) | CNF Time 0.5000(0.5000)
Iter 15496 | Time 0.7901(0.7525) | Loss -2.887002(-2.426352) |

Iter 15551 | Time 0.7713(0.7701) | Loss -2.985030(-2.452254) | NFE Forward 62(60.3) | NFE Backward 86(86.8) | CNF Time 0.5000(0.5000)
Iter 15552 | Time 0.7586(0.7693) | Loss -1.289514(-2.370862) | NFE Forward 62(60.4) | NFE Backward 86(86.7) | CNF Time 0.5000(0.5000)
Iter 15553 | Time 0.7372(0.7670) | Loss -1.944383(-2.341009) | NFE Forward 56(60.1) | NFE Backward 86(86.7) | CNF Time 0.5000(0.5000)
Iter 15554 | Time 0.7746(0.7675) | Loss -2.020394(-2.318566) | NFE Forward 62(60.2) | NFE Backward 86(86.6) | CNF Time 0.5000(0.5000)
Iter 15555 | Time 0.7768(0.7682) | Loss -2.484221(-2.330162) | NFE Forward 62(60.4) | NFE Backward 86(86.6) | CNF Time 0.5000(0.5000)
Iter 15556 | Time 0.7769(0.7688) | Loss -2.152596(-2.317732) | NFE Forward 62(60.5) | NFE Backward 86(86.5) | CNF Time 0.5000(0.5000)
Iter 15557 | Time 0.7753(0.7693) | Loss -2.769066(-2.349325) | NFE Forward 62(60.6) | NFE Backward 86(86.5) | CNF Time 0.5000(0.5000)
Iter 15558 | Time 0.7734(0.7696) | Loss -2.730883(-2.376034) |

Iter 15612 | Time 0.7747(0.7671) | Loss -3.047558(-2.473622) | NFE Forward 62(59.7) | NFE Backward 86(86.0) | CNF Time 0.5000(0.5000)
Iter 15613 | Time 0.7304(0.7645) | Loss -1.832970(-2.428776) | NFE Forward 62(59.9) | NFE Backward 86(86.0) | CNF Time 0.5000(0.5000)
Iter 15614 | Time 0.7742(0.7652) | Loss -3.201918(-2.482896) | NFE Forward 62(60.0) | NFE Backward 86(86.0) | CNF Time 0.5000(0.5000)
Iter 15615 | Time 0.8355(0.7701) | Loss -1.869833(-2.439982) | NFE Forward 62(60.2) | NFE Backward 98(86.8) | CNF Time 0.5000(0.5000)
Iter 15616 | Time 0.7745(0.7704) | Loss -2.430072(-2.439288) | NFE Forward 62(60.3) | NFE Backward 86(86.8) | CNF Time 0.5000(0.5000)
Iter 15617 | Time 0.8121(0.7733) | Loss -2.560044(-2.447741) | NFE Forward 62(60.4) | NFE Backward 92(87.2) | CNF Time 0.5000(0.5000)
Iter 15618 | Time 0.7904(0.7745) | Loss -1.374173(-2.372591) | NFE Forward 68(60.9) | NFE Backward 86(87.1) | CNF Time 0.5000(0.5000)
Iter 15619 | Time 0.7869(0.7754) | Loss -2.844730(-2.405641) |

Iter 15674 | Time 0.7371(0.7576) | Loss -2.293433(-2.388336) | NFE Forward 56(60.9) | NFE Backward 86(85.3) | CNF Time 0.5000(0.5000)
Iter 15675 | Time 0.7693(0.7584) | Loss -1.605345(-2.333527) | NFE Forward 62(61.0) | NFE Backward 86(85.4) | CNF Time 0.5000(0.5000)
Iter 15676 | Time 0.8184(0.7626) | Loss -2.097377(-2.316996) | NFE Forward 62(61.1) | NFE Backward 98(86.3) | CNF Time 0.5000(0.5000)
Iter 15677 | Time 0.8621(0.7695) | Loss -2.599179(-2.336749) | NFE Forward 62(61.1) | NFE Backward 98(87.1) | CNF Time 0.5000(0.5000)
Iter 15678 | Time 0.8066(0.7721) | Loss -1.965859(-2.310787) | NFE Forward 62(61.2) | NFE Backward 92(87.4) | CNF Time 0.5000(0.5000)
Iter 15679 | Time 0.7709(0.7720) | Loss -2.853180(-2.348754) | NFE Forward 62(61.3) | NFE Backward 86(87.3) | CNF Time 0.5000(0.5000)
Iter 15680 | Time 0.7785(0.7725) | Loss -2.017648(-2.325577) | NFE Forward 62(61.3) | NFE Backward 86(87.2) | CNF Time 0.5000(0.5000)
Iter 15681 | Time 0.7366(0.7700) | Loss -2.265643(-2.321381) |

Iter 15736 | Time 0.7732(0.7694) | Loss -2.798290(-2.454490) | NFE Forward 62(60.1) | NFE Backward 86(85.6) | CNF Time 0.5000(0.5000)
Iter 15737 | Time 0.7766(0.7699) | Loss -2.337574(-2.446306) | NFE Forward 62(60.2) | NFE Backward 86(85.6) | CNF Time 0.5000(0.5000)
